In [1]:
import numpy as np
import pandas as pd
import open3d
import os
import tqdm
import utils.helpers as helpers
import utils.fread as fread
import utils.registration as registration
import scipy

from utils.depth_camera import DepthCamera

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [62]:
data_dir = "data/exp_7"
trial = "trial_1"
subject = "subject-1"
sequence = 1

In [63]:
motion_dir = os.path.join(data_dir, trial, subject, f"{sequence:02d}", "motion")

In [64]:
accel_df = pd.read_csv(os.path.join(motion_dir, "accelerometer"))
gyro_df = pd.read_csv(os.path.join(motion_dir, "gyroscope"))

In [65]:
alpha = 0.9
gravity = [0, 0, 0]

for i in tqdm.trange(accel_df.shape[0]):
    for j in range(3):
        gravity[j] = alpha * gravity[j] + (1 - alpha) * accel_df.iloc[i, 1 + j]
        accel_df.iloc[i, 1 + j] = accel_df.iloc[i, 1 + j] - gravity[j]

100%|██████████| 11964/11964 [00:02<00:00, 4182.99it/s]


In [66]:
camera = DepthCamera("lidar", "data/exp_7/metadata/device-0-aligned.json")

In [67]:
sequence_dir = os.path.join(data_dir, trial, subject, f"{sequence:02d}", "frames")
sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")

In [68]:
start_t = sequence_ts[0]
end_t = sequence_ts[-1]

In [69]:
accel_t = accel_df[(accel_df.timestamp >= start_t) & (accel_df.timestamp <= end_t)]
# remove redundant data
accel_t = accel_t.drop_duplicates("timestamp")
# calculating time difference
# accel_t.loc[:, "dt"] = np.concatenate([[0], (accel_t.timestamp.values[1:] - accel_t.timestamp.values[:-1]) / 1000])
accel_t.loc[:, "dt"] = (accel_t.timestamp.values - accel_t.timestamp.values[0]) / 1000
# remove first row
accel_t = accel_t.iloc[1:]

In [70]:
accel_t.loc[:, "Ix"] = scipy.integrate.cumtrapz(accel_t.x.values, accel_t.dt.values, initial=0)
accel_t.loc[:, "Iy"] = scipy.integrate.cumtrapz(accel_t.y.values, accel_t.dt.values, initial=0)
accel_t.loc[:, "Iz"] = scipy.integrate.cumtrapz(accel_t.z.values, accel_t.dt.values, initial=0)

In [71]:
accel_t.loc[:, "I2x"] = scipy.integrate.cumtrapz(accel_t.Ix.values, accel_t.dt.values, initial=0)
accel_t.loc[:, "I2y"] = scipy.integrate.cumtrapz(accel_t.Iy.values, accel_t.dt.values, initial=0)
accel_t.loc[:, "I2z"] = scipy.integrate.cumtrapz(accel_t.Iz.values, accel_t.dt.values, initial=0)

In [72]:
# t = accel_t.I2x.values, accel_t.I2y.sum(), accel_t.I2z.sum()
t = accel_t.values[-1, -3:]

In [73]:
gyro_t = gyro_df[(accel_df.timestamp >= start_t) & (accel_df.timestamp <= end_t)]
# remove redundant data
gyro_t = gyro_t.drop_duplicates("timestamp")
# calculating time difference
# gyro_t.loc[:, "dt"] = np.concatenate([[0], (gyro_t.timestamp.values[1:] - gyro_t.timestamp.values[:-1]) / 1000])
gyro_t.loc[:, "dt"] = (gyro_t.timestamp.values - gyro_t.timestamp.values[0]) / 1000
# remove first row
gyro_t = gyro_t.iloc[1:]

In [74]:
gyro_t.loc[:, "Ix"] = scipy.integrate.cumtrapz(gyro_t.x.values, gyro_t.dt.values, initial=0)
gyro_t.loc[:, "Iy"] = scipy.integrate.cumtrapz(gyro_t.y.values, gyro_t.dt.values, initial=0)
gyro_t.loc[:, "Iz"] = scipy.integrate.cumtrapz(gyro_t.z.values, gyro_t.dt.values, initial=0)

In [75]:
# R = gyro_t.Ix.sum(), gyro_t.Iy.sum(), gyro_t.Iz.sum()
R = gyro_t.values[-1, -3:]

In [76]:
T = helpers.rotate_transformation_matrix(np.identity(4), R[0], R[1], R[2])
T[:3, 3] = t

In [77]:
print(f"Translation: {t}", end="\t")
print(f"Rotation: {R}")

Translation: [-0.01919421  0.00721959  0.02827663]	Rotation: [0.02045962 2.10212987 0.4097772 ]


In [78]:
source = camera.depth_to_point_cloud(os.path.join(sequence_dir, f"frame-{start_t}.depth.png"))
target = camera.depth_to_point_cloud(os.path.join(sequence_dir, f"frame-{end_t}.depth.png"))

In [79]:
registration.view(source, target, T)

In [23]:
accel_df = accel_df.drop_duplicates("timestamp")

In [24]:
samples = accel_df.shape[0]
duration = accel_df.timestamp.values[-1] - accel_df.timestamp.values[0]

In [26]:
samples / duration * 1000

401.9453652399042